<a href="https://colab.research.google.com/github/hanhaotian/nlp-tutorial/blob/master/nlp-tutorial/1-1.NNLM/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

dtype = torch.FloatTensor # FloatTensor类型

sentences = ['I like milk', 'He love apple', 'She hate banana']

# 输入层的词表|V|，即语料库中出现过的所有唯一词汇数量
vocab_list = set(sorted([i for x in sentences for i in x.split(' ')]))  
word_2_index = {w: i for i, w in enumerate(vocab_list)}
index_2_word = {i: w for i, w in enumerate(vocab_list)}

V = len(vocab_list) # 词表的大小
n = len(sentences[0].split(' ')) - 1  # 窗口大小
h = 3 # 隐藏层大小
m = 3 # 特征大小

class NNLM(nn.Module):
  def __init__(self, V, n, h, m):
    super().__init__()
    # torch.nn.Embedding(num_embeddings, embedding_dim)
    # a. 保存了固定字典和大小的简单查找表，通常用于保存词嵌入和用下标检索它们
    # b. 用于创建一个词嵌入模型，num_embeddings代表一共有多少个词，embedding_dim表示为每个词创建多少维的向量
    self.C = nn.Embedding(V, m)
    print('C: ', self.C)
    self.H = nn.Parameter(torch.zeros(m * n, h))
    print('H: ', self.H)
    self.d = nn.Parameter(torch.zeros(h))
    print('d: ', self.d.size())
    self.U = nn.Parameter(torch.zeros(h, V))
    self.W = nn.Parameter(torch.zeros(n*m, V))
    self.b = nn.Parameter(torch.zeros(V))

  def forward(self, input):
    x = self.C(input) # 3 x 2 x 3 每个词转换为指定维度
    x = x.view(-1, n * m) # 连接操作 => 3 x 6
    hidden = torch.tanh(torch.mm(x, self.H) + self.d) # torch.mm表示矩阵相乘 => 3 x 6 * 6 x 3 + 3 x 1
    y = self.b + torch.mm(x, self.W) + torch.mm(hidden, self.U)
    output = torch.softmax(y, 1)
    return output

input_batch = []
output_batch = []

for s in sentences:
  words = s.split(' ')
  input_batch.append([word_2_index[i] for i in words[:-1]])
  output_batch.append(word_2_index[words[-1]])

print(word_2_index)


input_batch = torch.LongTensor(input_batch) # 3 x 2
output_batch = torch.LongTensor(output_batch) # 3 x 1

print(input_batch.size())
print(output_batch.size())

model = NNLM(V, n, h, m)

# 神经网络优化器，主要是为了优化我们的神经网络，使它在我们训练过程中快起来，节省训练网络的时间
# 优化器是需要根据网络反向传播的梯度信息来更新网络的参数，以起到降低loss函数计算值的作用。
# a. 优化器需要知道当前网络或其他模型的参数空间，这也是为什么训练文件中，正式开始训练前，需要将网络的参数放到优化器里面
# b. 需要知道反向传播的梯度信息
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # 学习率为0.01
# 交叉熵损失函数
criterion = torch.nn.CrossEntropyLoss()

loss_list = []
for epoch in range(1001):
  output = model(input_batch)
  # 计算损失函数
  loss = criterion(output, output_batch)
  # 将模型的参数梯度初始化为0，根据pythorch的backward函数计算，当网络参量进行反馈时，梯度是累积计算而不是被替换，但在处理每一个batch
  # 时并不需要与其他batch的梯度混合起来累积计算。因此需要对每一个batch调用一遍zero_grad()将参数梯度置为0
  optimizer.zero_grad() # 梯度设置为0
  loss.backward() # 反向传播
  optimizer.step()  # 用来更新优化器的学习率，一般按照epoch为单位进行更换，即多少个epoch后更新一次学习率，
  loss_list.append(loss.data.numpy())

  if epoch % 200 == 0:
    print('Epoch {}: {}'.format(epoch, loss))
    print('model.C: ', model.C)
    print('model.H: ', model.H)
    print('model.d: ', model.d)
    print('model.U: ', model.U)
    print('model.W: ', model.W)
    print('\n\n')


for index, input in enumerate(input_batch):
  predict = model(input)
  input_word = [index_2_word[int(index)] for index in input]
  print('-'*40)
  print('Predict: ', ' '.join(input_word), '->', index_2_word[int(torch.argmax(predict, dim=1))], '\nTruth', sentences[index])


{'love': 0, 'apple': 1, 'like': 2, 'I': 3, 'milk': 4, 'He': 5, 'hate': 6, 'banana': 7, 'She': 8}
torch.Size([3, 2])
torch.Size([3])
C:  Embedding(9, 3)
H:  Parameter containing:
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True)
d:  torch.Size([3])
Epoch 0: 2.1972246170043945
model.C:  Embedding(9, 3)
model.H:  Parameter containing:
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True)
model.d:  Parameter containing:
tensor([0., 0., 0.], requires_grad=True)
model.U:  Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True)
model.W:  Parameter containing:
tensor([[ 1.4170e-04, -9.2302e-04,  1.4170e-04,  1.4170e-04,  6.0657e-05,
          1.4170e-04,  1.4170e-04,  1.21

In [0]:
# 类型转换，将list、numpy转化为tensor
print(torch.FloatTensor([1, 2]))
# 根据torch.size()创建一个tensor
a = torch.tensor([[1, 2], [3, 4]])
print(torch.FloatTensor(a.size()))

In [0]:
'''
nn.Embedding测试1
'''
from torch.autograd import Variable
word_2_ix = {'hello': 0, 'word': 1}
embeds = nn.Embedding(2, 5) # 这里的词向量的建立知识初始的词向量，并没有经过任何修改优化
print(embeds)
hello_idx = Variable(torch.LongTensor([word_2_ix['hello']])) # 得到一个Variable，值为hello的索引0
print(hello_idx)
print(embeds(hello_idx))  # 得到word embedding中关于hello这个词的初始词向量
word_idx = Variable(torch.LongTensor([word_2_ix['word']]))
print(word_idx)
print(embeds(word_idx))

'''
nn.Embedding测试2
'''
embeds = nn.Embedding(5, 4) # 假定字典中只有5个词，词向量维度为4
# 每个数字代表一个词，如{'!': 0, 'how': 1, 'are': 2, 'you': 3, 'ok':4}，且这些数字的范围只能在0-4之间，因为上面定义了只有5个词
word = [[1, 2, 3], [2, 3, 4]]
embed = embeds(torch.LongTensor(word)) 
print(embed)
print(embed.size()) # torch.Size([2, 3, 4])表示对于输入的[2, 3]维的词，每一个词都被映射成了一个4维的向量

'''
torch.zeros测试
'''
print(torch.zeros(3))

'''
tensor.view变换
'''
# view返回一个有相同数据但大小不同的tensor，即进行reshape操作
x = torch.randn(3, 4, 5, 7)
print('tensor原型:' , x)
# -1表示剩下的值的个数一起构成一个维度，例如，第一个参数1将第一个维度设定为1，后一个-1即第二个维度的大小=元素总数目/第一个维度的大小，此例为3*4*5*7/1=420
b = x.view(1, -1)
print(b.size())
d = x.view(x.size(0), x.size(1), -1) # 3*4*5*7/(3*4)=35
print(d.size())
e = x.view(4, -1, 5) # 3*4*5*7/(4*5) = 21
print(e.size())
